In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [ ]:
!cp ./drive/My\ Drive/0910_b_test.csv ./
!cp ./drive/My\ Drive/0910_b_train.csv ./

In [7]:
!top

=top - 09:25:33 up 16 min,  0 users,  load average: 0.05, 0.11, 0.09
Tasks:  11 total,   1 running,   9 sleeping,   0 stopped,   1 zombie
%Cpu(s):  3.3 us,  1.6 sy,  0.0 ni, 93.7 id,  1.4 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 13333536 total,  9708960 free,   884244 used,  2740332 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 12193740 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND   
      8 root      20   0  309724  55940  31164 S   6.7  0.4   0:02.72 node      
    303 root      20   0  197516  63636  13952 S   6.7  0.5   0:02.90 jupyter-+ 
      1 root      20   0   39192   6388   4884 S   0.0  0.0   0:00.27 run.sh    
     94 root      20   0   35888   4876   3784 S   0.0  0.0   0:00.41 tail      
    137 root      20   0   18376   1576   1272 S   0.0  0.0   0:00.00 bash      
    138 root      20   0 1120216  19392  16556 S   0.0  0.1   0:00.02 drive     
    139 root      20   0   11592   2068   1844 S   0.0  0.0  

In [ ]:
!kill -9  303

In [6]:
!cp ./checkpoint ./drive/My\ Drive/Colab\ Notebooks/logs

In [2]:
import numpy as np
import tensorflow as tf
import time
import csv
from random import shuffle
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import metrics
from math import sqrt
#第一处修改
# FLAGS = tf.app.flags.FLAGS
# for attr, value in sorted(FLAGS.__flags.items()): 
#   FLAGS.__delattr__(attr.upper())

# FLAGS = tf.app.flags.FLAGS
# FLAGS.flag_values_dict()
# for attr in ['f',"epsilon", "l2_lambda","learning_rate","max_grad_norm","max_grad_norm", "keep_prob","hidden_layer_num","hidden_size","evaluation_interval","batch_size", "epochs","allow_soft_placement","log_device_placement","train_data_path","test_data_path"]:
#   if attr in FLAGS.flag_values_dict():
#     FLAGS.__delattr__(attr)


# flags
tf.flags.DEFINE_float("epsilon", 0.1, "Epsilon value for Adam Optimizer.")
tf.flags.DEFINE_float("l2_lambda", 0.3, "Lambda for l2 loss.")
tf.flags.DEFINE_float("learning_rate", 0.1, "Learning rate")
tf.flags.DEFINE_float("max_grad_norm", 20.0, "Clip gradients to this norm.")
tf.flags.DEFINE_float("keep_prob", 0.6, "Keep probability for dropout")
tf.flags.DEFINE_integer("hidden_layer_num", 1, "The number of hidden layers (Integer)")
tf.flags.DEFINE_integer("hidden_size", 200, "The number of hidden nodes (Integer)")
tf.flags.DEFINE_integer("evaluation_interval", 3, "Evaluate and print results every x epochs")
tf.flags.DEFINE_integer("batch_size", 32, "Batch size for training.")
tf.flags.DEFINE_integer("epochs", 10, "Number of epochs to train for.")
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.flags.DEFINE_string("train_data_path", './0910_b_train.csv', "Path to the training dataset")
tf.flags.DEFINE_string("test_data_path", './0910_b_test.csv', "Path to the testing dataset")


FLAGS = tf.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel') #第二处修改
FLAGS.flag_values_dict() #第三处修改 FLAGS._parse_flags()

print("\nParameters:")
for attr, value in sorted(FLAGS.flag_values_dict().items()):  #FLAGS.flag_values_dict().items()
    print("{}={}".format(attr.upper(), value))
print("")

def add_gradient_noise(t, stddev=1e-3, name=None):
    """
    Adds gradient noise as described in http://arxiv.org/abs/1511.06807 [2].

    The input Tensor `t` should be a gradient.

    The output will be `t` + gaussian noise.

    0.001 was said to be a good fixed value for memory networks [2].
    """
    #WARNING:tensorflow:tf.op_scope(values, name, default_name) is deprecated, use tf.name_scope(name, default_name, values)
    with tf.op_scope([t, stddev], name, "add_gradient_noise") as name:
        t = tf.convert_to_tensor(t, name="t")
        gn = tf.random_normal(tf.shape(t), stddev=stddev)
        return tf.add(t, gn, name=name)

class StudentModel(object):

    def __init__(self, is_training, config):
        self._batch_size = batch_size = FLAGS.batch_size
        self.num_skills = num_skills = config.num_skills
        self.hidden_size = size = FLAGS.hidden_size
        self.num_steps = num_steps = config.num_steps #序列长度
        input_size = num_skills*2

        inputs = self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) # x[i, j] = label_index
        self._target_id = target_id = tf.placeholder(tf.int32, [None])
        self._target_correctness = target_correctness = tf.placeholder(tf.float32, [None])
        final_hidden_size = size

        hidden_layers = []
        for i in range(FLAGS.hidden_layer_num):
            final_hidden_size = int(size/(i+1))
            hidden1 = tf.nn.rnn_cell.LSTMCell(final_hidden_size, state_is_tuple=True)
            if is_training and config.keep_prob < 1:
                hidden1 = tf.nn.rnn_cell.DropoutWrapper(hidden1, output_keep_prob=FLAGS.keep_prob)
            hidden_layers.append(hidden1)
        # hidden1包含DropoutWrapper和LSTMCell
        cell = tf.nn.rnn_cell.MultiRNNCell(hidden_layers, state_is_tuple=True)

        input_data = tf.reshape(self._input_data, [-1]) #reshape成一个维度
        #one-hot encoding
        with tf.device("/gpu:0"):
            labels = tf.expand_dims(input_data, 1) # [batch_size*num_steps] -> [batch_size*num_steps,1]
            indices = tf.expand_dims(tf.range(0, batch_size*num_steps, 1), 1) #tf.range用于创建数字序列变量（start，limit，delta）
            concated = tf.concat([indices, labels],1) # [batch_size*num_steps,2]

            #真正的input形状
            inputs = tf.sparse_to_dense(concated, tf.stack([batch_size*num_steps, input_size]), 1.0, 0.0) #输入为one-hot矩阵 参数：sparse_indices、output_shape、sparse_values、default_value
            inputs.set_shape([batch_size*num_steps, input_size])

        # [batch_size, num_steps, input_size]
        inputs = tf.reshape(inputs, [-1, num_steps, input_size])
        x = tf.transpose(inputs, [1, 0, 2])
        # Reshape to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, input_size])
        # Split to get a list of 'n_steps'
        # tensors of shape (doc_num, n_input)
        x = tf.split(x, num_steps, 0) #split_dim,num_split，value：The `Tensor` to split.

        # inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # outputs, state = tf.nn.rnn(hidden1, x, dtype=tf.float32)

        # tf.nn.rnn
        #inputs – A length T list of inputs, each a `Tensor` of shape `[batch_size, input_size]
        #sequence_length – Specifies the length of each sequence in inputs. An int32 or int64 vector (tensor) size `[batch_size]`, values in `[0, T)`.
        #output, state = cell(input_, state) outputs.append(output) return (outputs, state) outputs为list
        outputs, state = tf.nn.static_rnn(cell, x, dtype=tf.float32)
        output = tf.reshape(tf.concat(outputs,1), [-1, int(final_hidden_size)]) #outputs列表里面的元素为[batch , input_size] 所以concat的是input_size #reshape成[batch * num_steps , input_size]
        # calculate the logits from last hidden layer to output layer
        sigmoid_w = tf.get_variable("sigmoid_w", [final_hidden_size, num_skills])
        sigmoid_b = tf.get_variable("sigmoid_b", [num_skills])
        logits = tf.matmul(output, sigmoid_w) + sigmoid_b #[batch * num_steps , num_skills]

        # from output nodes to pick up the right one we want
        logits = tf.reshape(logits, [-1])
        selected_logits = tf.gather(logits, self.target_id) # 哪个batch的第几道题目所涉及的技能的预测——单个元素 总共有batch * problem_ids个

        #make prediction
        self._pred = self._pred_values = pred_values = tf.sigmoid(selected_logits)

        # loss function
        loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels = target_correctness , logits = selected_logits))
        
        #self._cost = cost = tf.reduce_mean(loss)
        self._cost = cost = loss

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def input_data(self):
        return self._input_data

    @property
    def auc(self):
        return self._auc

    @property
    def pred(self):
        return self._pred

    @property
    def target_id(self):
        return self._target_id

    @property
    def target_correctness(self):
        return self._target_correctness

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def pred_values(self):
        return self._pred_values

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state

class HyperParamsConfig(object):
    """Small config."""
    init_scale = 0.05
    num_steps = 0
    max_grad_norm = FLAGS.max_grad_norm
    max_max_epoch = FLAGS.epochs
    keep_prob = FLAGS.keep_prob
    num_skills = 0

from tqdm import tqdm
def run_epoch(session, m, students, eval_op, verbose=False):
    """Runs the model on the given data."""
    start_time = time.time()

    index = 0
    pred_labels = []
    actual_labels = []
    total = len(students)

    pbar = tqdm(range(total))

    while(index+m.batch_size < len(students)):
        x = np.zeros((m.batch_size, m.num_steps))
        target_id = []
        target_correctness = []
        count = 0
        for i in range(m.batch_size):
            student = students[index+i]
            problem_ids = student[1]
            correctness = student[2]
            for j in range(len(problem_ids)-1): # j只能渠道len-2
                problem_id = int(problem_ids[j])
                label_index = 0
                if(int(correctness[j]) == 0):
                    label_index = problem_id
                else:
                    label_index = problem_id + m.num_skills #m.num_skills = config.num_skills = train_max_skill_num
                x[i, j] = label_index
                target_id.append(i * m.num_steps * m.num_skills + j * m.num_skills + int(problem_ids[j+1]) ) #索引对象：[batch * num_steps , num_skills]
                target_correctness.append(int(correctness[j+1])) #j range from 1 to len(problem_ids)-1
                actual_labels.append(int(correctness[j+1]))

        index += m.batch_size
        # print("current/total: {} : {}".format(index,total),end="\r") 在pycharm没用
        pbar.update(m.batch_size)

        pred, _ = session.run([m.pred, eval_op], feed_dict={
            m.input_data: x, m.target_id: target_id,
            m.target_correctness: target_correctness})  # m.pred：make prediction   eval_op：optimizer.apply_gradients里面用到m.cost

        for p in pred:
            pred_labels.append(p)
    #print pred_labels
    rmse = sqrt(mean_squared_error(actual_labels, pred_labels)) # pred_labels经过sigmoid函数
    fpr, tpr, thresholds = metrics.roc_curve(actual_labels, pred_labels, pos_label=1)
    auc = metrics.auc(fpr, tpr)

    #calculate r^2
    r2 = r2_score(actual_labels, pred_labels)
    return rmse, auc, r2


def read_data_from_csv_file(fileName):
    config = HyperParamsConfig()
    inputs = []
    targets = []
    rows = []
    max_skill_num = 0
    max_num_problems = 0
    with open(fileName, "r") as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            rows.append(row)
    index = 0
    i = 0
    print ("the number of rows is " + str(len(rows)))
    tuple_rows = []
    #turn list to tuple
    while(index < len(rows)-1): # index += 3
        problems_num = int(rows[index][0])
        tmp_max_skill = max(map(int, rows[index+1]))
        if(tmp_max_skill > max_skill_num):
            max_skill_num = tmp_max_skill
        if(problems_num <= 2):
            index += 3
        else:
            if problems_num > max_num_problems:
                max_num_problems = problems_num
            tup = (rows[index], rows[index+1], rows[index+2])
            tuple_rows.append(tup)
            index += 3
    #shuffle the tuple
    random.seed(10101)
    random.shuffle(tuple_rows)
    print ("The number of students is ", len(tuple_rows))
    print ("Finish reading data")
    return tuple_rows, max_num_problems, max_skill_num+1

def main(unused_args):
    config = HyperParamsConfig()
    eval_config = HyperParamsConfig()
    # timestamp = str(time.time())
    timestamp = time.strftime("%m-%d %H:%M", time.localtime())
    train_data_path = FLAGS.train_data_path
    #path to your test data set
    test_data_path = FLAGS.test_data_path
    #the file to store your test results
    result_file_path = "run_logs_{}".format(timestamp)
    #your model name
    model_name = "./DKT"

    train_students, train_max_num_problems, train_max_skill_num = read_data_from_csv_file(train_data_path)
    config.num_steps = train_max_num_problems
    config.num_skills = train_max_skill_num

    test_students, test_max_num_problems, test_max_skill_num = read_data_from_csv_file(test_data_path)
    eval_config.num_steps = test_max_num_problems
    eval_config.num_skills = test_max_skill_num

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement,
                                      log_device_placement=FLAGS.log_device_placement)

        global_step = tf.Variable(0, name="global_step", trainable=False)
        # decay learning rate
        starter_learning_rate = FLAGS.learning_rate
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 3000, 0.96, staircase=True)

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=FLAGS.epsilon)

        with tf.Session(config=session_conf) as session:

            initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)

            # training model
            with tf.variable_scope("model", reuse=None, initializer=initializer):
                m = StudentModel(is_training=True, config=config)
            # testing model
            with tf.variable_scope("model", reuse=True, initializer=initializer):
                mtest = StudentModel(is_training=False, config=eval_config)

            grads_and_vars = optimizer.compute_gradients(m.cost) #m.cost = loss  Returns:A list of (gradient, variable) pairs. var_list:Optional list of `tf.Variable` to update to minimize `loss`
            grads_and_vars = [(tf.clip_by_norm(g, FLAGS.max_grad_norm), v)
                              for g, v in grads_and_vars if g is not None]
            grads_and_vars = [(add_gradient_noise(g), v) for g, v in grads_and_vars]
            train_op = optimizer.apply_gradients(grads_and_vars, name="train_op", global_step=global_step) #global_step – Optional `Variable` to increment by one after the variables have been updated.
            session.run(tf.global_variables_initializer()) # tf.initialize_all_variables()
            # log hyperparameters to results file
            with open(result_file_path, "a+") as f:
                print("Writing hyperparameters into file")
                f.write("Hidden layer size: %d \n" % (FLAGS.hidden_size))
                f.write("Dropout rate: %.3f \n" % (FLAGS.keep_prob))
                f.write("Batch size: %d \n" % (FLAGS.batch_size))
                f.write("Max grad norm: %d \n" % (FLAGS.max_grad_norm))
            saver = tf.train.Saver(tf.global_variables()) #tf.all_variables()

            for i in range(config.max_max_epoch):
                rmse, auc, r2 = run_epoch(session, m, train_students, train_op, verbose=True)
                print("Epoch: %d Train Metrics:\n rmse: %.3f \t auc: %.3f \t r2: %.3f \n" % (i + 1, rmse, auc, r2))

                if((i+1) % FLAGS.evaluation_interval == 0):
                    print ("Save variables to disk")
                    save_path = saver.save(session, model_name + str(i))
                    print("*"*10)
                    print("Start to test model....")
                    rmse, auc, r2 = run_epoch(session, mtest, test_students, tf.no_op())
                    print("Epoch: %d Test Metrics:\n rmse: %.3f \t auc: %.3f \t r2: %.3f" % (i+1, rmse, auc, r2))
                    with open(result_file_path, "a+") as f:
                        f.write("Epoch: %d Test Metrics:\n rmse: %.3f \t auc: %.3f \t r2: %.3f" % ((i+1), rmse, auc, r2))
                        f.write("\n")

                        print("*"*10)

if __name__ == "__main__":
    tf.app.run()
  # main()


Parameters:
ALLOW_SOFT_PLACEMENT=True
ALSOLOGTOSTDERR=False
BATCH_SIZE=32
EPOCHS=10
EPSILON=0.1
EVALUATION_INTERVAL=3
F=
HIDDEN_LAYER_NUM=1
HIDDEN_SIZE=200
KEEP_PROB=0.6
L2_LAMBDA=0.3
LEARNING_RATE=0.1
LOG_DEVICE_PLACEMENT=False
LOG_DIR=
LOGGER_LEVELS={}
LOGTOSTDERR=False
MAX_GRAD_NORM=20.0
ONLY_CHECK_ARGS=False
OP_CONVERSION_FALLBACK_TO_WHILE_LOOP=False
PDB=False
PDB_POST_MORTEM=False
PROFILE_FILE=None
RUN_WITH_PDB=False
RUN_WITH_PROFILING=False
SHOWPREFIXFORINFO=True
STDERRTHRESHOLD=fatal
TEST_DATA_PATH=./0910_b_test.csv
TEST_RANDOM_SEED=301
TEST_RANDOMIZE_ORDERING_SEED=
TEST_SRCDIR=
TEST_TMPDIR=/tmp/absl_testing
TRAIN_DATA_PATH=./0910_b_train.csv
USE_CPROFILE_FOR_PROFILING=True
V=-1
VERBOSITY=-1
XML_OUTPUT_FILE=

the number of rows is 10116
The number of students is  3134
Finish reading data
the number of rows is 2532
The number of students is  786
Finish reading data
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Te

W1025 09:39:33.035723 139651713361792 deprecation.py:323] From <ipython-input-2-a30abf48945f>:82: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


W1025 09:39:33.044630 139651713361792 deprecation.py:323] From <ipython-input-2-a30abf48945f>:87: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


...

Epoch: 9 Test Metrics:
 rmse: 0.394 	 auc: 0.818 	 r2: 0.308
**********














  2%|▏         | 64/3134 [00:00<00:18, 164.20it/s]











  3%|▎         | 96/3134 [00:00<00:24, 124.96it/s]











  4%|▍         | 128/3134 [00:01<00:28, 106.83it/s]











  5%|▌         | 160/3134 [00:01<00:31, 95.57it/s] 











  6%|▌         | 192/3134 [00:01<00:31, 91.94it/s]











  7%|▋         | 224/3134 [00:02<00:32, 90.02it/s]











  8%|▊         | 256/3134 [00:02<00:32, 88.58it/s]











  9%|▉         | 288/3134 [00:03<00:32, 87.90it/s]











 10%|█         | 320/3134 [00:03<00:32, 86.03it/s]











 11%|█         | 352/3134 [00:03<00:33, 84.29it/s]











 12%|█▏        | 384/3134 [00:04<00:32, 83.73it/s]











 13%|█▎        | 416/3134 [00:04<00:33, 81.30it/s]











 14%|█▍        | 448/3134 [00:05<00:32, 81.56it/s]











 15%|█▌        | 480/3134 [00:05<00:32, 81.38it/s]











 16%|█▋        | 512/3134 [00:05<00:32, 81.20it/s]











 17%|█▋        | 544/3134 [00:06<00:31, 81.68it/s]







Epoch: 10 Train Metrics:
 rmse: 0.387 	 auc: 0.835 	 r2: 0.337 



SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
